In [2]:
import os
import requests
import time
import pandas as pd
import numpy as np
import torch 
from torch import nn
from torch.optim import AdamW  
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd 
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, AutoTokenizer, pipeline, BertTokenizer, BertModel
from transformers import Trainer, TrainingArguments
import spacy
from spacy import displacy

from sklearn.model_selection import train_test_split
from tqdm import tqdm
torch.set_default_device("cpu")
import random
import json

In [3]:
pwd

'/Users/chienshiyun/Documents/_DSA3101/proj/dsa3101/notebooks'

In [60]:
df_combined = pd.read_csv('../files/combined_pdfs_2602.csv')

In [ ]:
dfs = {}
pdf_folder_path = "datasets/"

for folder_name in os.listdir(pdf_folder_path):
    folder_path = os.path.join(pdf_folder_path, folder_name)  
    if os.path.isdir(folder_path):  
        json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        for file in json_files:
            file_path = os.path.join(folder_path, file)
            print(f"Processing file: {file_path}")
            
            file_path = os.path.join(folder_path, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)  # Load JSON file
            if isinstance(data, list):  
                df = pd.DataFrame(data, columns=["esg_text"])
            else:
                print(f"Skipping {file}: Unsupported format")
                continue
            
            dfs[file] = df
            
for file, df in dfs.items():
    match = re.match(r"(.+?)_(\d{4})_ocr.json", file)  
    if match:
        company_name, year = match.groups()
        df["Company"] = company_name
        df["Year"] = int(year)

# Concatenate all DataFrames into df_combined
df_combined = pd.concat(dfs.values(), ignore_index=True)
missing_rows = df_combined[df_combined.isnull().any(axis=1)]
print(len(missing_rows))
df_combined.drop_duplicates(inplace=True)

# Data Cleaning
1. Remove duplicate data 
2. Handle missing values

In [ ]:
with open('../dataset_json/Health/Na/Pfizer_2022_ocr.json', 'r', encoding='utf-8') as f:
    pfizer_data = json.load(f)  
pfizer_df = pd.DataFrame(pfizer_data)
pfizer_df.head(5)

In [ ]:
folder_path = "../dataset_json/Tech/AsiaPac/"

json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
dfs = {}

for file in json_files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)  # Load JSON file
    if isinstance(data, list):  
        df = pd.DataFrame(data)
    else:
        print(f"Skipping {file}: Unsupported format")
        continue
    
    dfs[file] = df  
dfs['pfizer_2022_ocr.json'] = pfizer_df

In [ ]:
folder_path = "../dataset_json/Tech/Na/"

json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
for file in json_files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)  # Load JSON file
    if isinstance(data, list):  
        df = pd.DataFrame(data)
    else:
        print(f"Skipping {file}: Unsupported format")
        continue
    
    dfs[file] = df  

In [ ]:
len(dfs)

In [ ]:
for df in dfs.values(): 
    print('df info is')
    #print(df.info())
    #print(df.describe(include="all") )
    #print(df.columns)
    #print(df.isnull().sum())
    print(df.duplicated().sum())

In [ ]:
df_combined = pd.concat([df for df in dfs.values()], ignore_index=True)  
missing_rows = df_combined[df_combined.isnull().any(axis=1)]
print(len(missing_rows))
df_combined.drop_duplicates(inplace=True)
df_combined["esg_text"].apply(type).value_counts()

In [ ]:
df_combined.drop_duplicates(inplace=True)
df_combined["esg_text"].apply(type).value_counts()

# Data Transformation

In [7]:
nlp = spacy.load("en_core_web_sm")

In [28]:
keywords = [
    "Environment", "Energy", "Waste", "Pollutants", "Water", "Climate",
    "Decarbonisation", "Carbon", "emissions", "Strategy", "Social", "Labor", "Practices", "Human Rights",
    "Human Capital", "Capital", "Management", "Occupational", "Health" ,"Safety", "Financial Inclusion",
    "Community investment", "Customer","Relations", "Privacy", "Protection",
    "Gender", "Women", "Female", "Race", "Ethnic", "Diversity", "Governance", "Transparency", "Reporting", "Corporate Governance",
    "Materiality", "Risk", "Crisis",  "Management", "Ethics", "Policy",
    "Tax Strategy", "Shareholder", "Information Security", "Cybersecurity", "System Availability","Sustainable", "Sustainable Finance", "Board Diversity"
]
keywords_regex = "|".join([f"\\b{k}\\b" for k in keywords])  # \b ensures whole-word matching


def extract_ner(text):
    if pd.isna(text):
        return None
    
    doc = nlp(text)
    entities = {"numbers": [], "metrics": [], "dates": []}

    for ent in doc.ents:
        if ent.label_ in ["CARDINAL", "ORDINAL"]:  # Numbers (e.g., "10", "zero", "first")
            entities["numbers"].append(ent.text)
        elif ent.label_ in ["QUANTITY", "PERCENT", "MONEY"]:  # Metrics (e.g., "5kg", "30%", "$100")
            entities["metrics"].append(ent.text)
        elif ent.label_ in ["DATE", "TIME"]:  # Dates/Time (e.g., "January 1, 2024", "3 PM")
            entities["dates"].append(ent.text)

    return entities if any(entities.values()) else None


df_combined["NER_Results"] = df_combined["esg_text"].apply(extract_ner)
df_filtered = df_combined[
    df_combined["NER_Results"].notna() |  #df_combined.dropna(subset=["NER_Results"])
    df_combined["esg_text"].str.contains(keywords_regex, case=False, na=False, regex=True)
]


In [33]:
print(len(df_combined))
print(len(df_filtered))

10702
7253


In [34]:
df_filtered.to_csv('../files/combined_pdfs_filteredNER_2802.csv', index=False)

In [35]:
df_filtered.head(5)

,esg_text,NER_Results
0,AOD MIOLS YALNNOD GWNY 766 AGOD MIOLS YA...,"{'numbers': ['4', '6', '8', '10', '11', '12', ..."
1,This is the 18th consecutive year we have publ...,"{'numbers': [], 'metrics': [], 'dates': ['the ..."
2,"Clearly, the past year presented our world wit...","{'numbers': [], 'metrics': [], 'dates': ['the ..."
3,It is clear that Al will be a major priority a...,"{'numbers': [], 'metrics': [], 'dates': ['year..."
8,Al for environmental sustainability We're pro...,"{'numbers': ['net-zero'], 'metrics': [], 'date..."


# Data labeling

In [61]:
model_name = "nbroad/ESG-BERT" #"nlpaueb/sec-bert-esg" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [63]:
candidate_labels = ["Environment - Energy efficiency", "Environment - Waste & Pollutants Generation", 
                    "Environment - Water Usage", "Environment - Climate Strategy", 
                    "Environment - Decarbonisation/Carbon emissions", "Environment - Strategy",
                    "Social - Labor Practices", "Social - Human Rights", "Social - Human Capital Management", 
                    "Social - Occupational Health & Safety", "Social - Financial Inclusion", "Social - Community investment",
                    "Social - Customer Relations","Social - Privacy Protection", 'Social - Gender and Ethnic Diversity',
                    "Governance - Transparency & Reporting", "Corporate Governance", "Governance - Materiality", 
                    "Governance - Risk & Crisis Management", "Governance - Business Ethics", 
                    "Governance - Policy Influence", "Governance - Tax Strategy", 
                    "Governance - Shareholder rights",
                    "Governance - Information Security/ Cybersecurity & System Availability", 
                    "Governance - Sustainable Finance", "Governance - Board Diversity"]

THRESHOLD=0.5
def classify_text(text):
    if pd.isna(text): 
        return None

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)  # Softmax for classification
    probabilities = probabilities.squeeze().cpu().numpy()

    if len(probabilities) != len(candidate_labels):
        print(f"Warning: Mismatch! Probabilities: {len(probabilities)}, Labels: {len(candidate_labels)}")
        return ["Error"]

    assigned_labels = [candidate_labels[i] for i, prob in enumerate(probabilities) if prob > THRESHOLD]
    return assigned_labels if assigned_labels else ["No Label"]

tqdm.pandas()


df_combined["labels"] = df_combined["esg_text"].apply(classify_text)
print(df_combined.head())


                                            esg_text      labels
0  AOD  MIOLS  YALNNOD  GWNY 766  AGOD  MIOLS  YA...  [No Label]
1  This is the 18th consecutive year we have publ...  [No Label]
2  Clearly, the past year presented our world wit...  [No Label]
3  It is clear that Al will be a major priority a...  [No Label]
4  We are committed to accelerating the deploymen...  [No Label]


In [64]:
df_combined.head(5)

,esg_text,labels
0,AOD MIOLS YALNNOD GWNY 766 AGOD MIOLS YA...,[No Label]
1,This is the 18th consecutive year we have publ...,[No Label]
2,"Clearly, the past year presented our world wit...",[No Label]
3,It is clear that Al will be a major priority a...,[No Label]
4,We are committed to accelerating the deploymen...,[No Label]


In [65]:
df_combined['labels'].value_counts()

labels
[No Label]                                                                  3587
[Social - Human Capital Management]                                          827
[Governance - Transparency & Reporting]                                      620
[Social - Customer Relations]                                                617
[Governance - Materiality]                                                   487
[Environment - Energy efficiency]                                            438
[Governance - Board Diversity]                                               412
[Governance - Sustainable Finance]                                           404
[Social - Financial Inclusion]                                               360
[Social - Occupational Health & Safety]                                      321
[Social - Privacy Protection]                                                303
[Environment - Waste & Pollutants Generation]                                293
[Governance - Busines

In [66]:
df_combined.to_csv('../labeled_pdfs_2802.csv', index=False)

# Model Training with ESG-BERT 

## Label Map

In [4]:
import ast
df = pd.read_csv('../files/labeled_pdfs_2802.csv')
df['labels'] = df['labels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [5]:
df.head(5)

,esg_text,labels
0,AOD MIOLS YALNNOD GWNY 766 AGOD MIOLS YA...,[No Label]
1,This is the 18th consecutive year we have publ...,[No Label]
2,"Clearly, the past year presented our world wit...",[No Label]
3,It is clear that Al will be a major priority a...,[No Label]
4,We are committed to accelerating the deploymen...,[No Label]


In [6]:
torch.manual_seed(42)
torch.mps.empty_cache()

device = (
    "mps" 
    if torch.backends.mps.is_available() 
    else "cuda" 
    if torch.cuda.is_available() 
    else "cpu"
)
device = torch.device(device)
print(f"Using device: {device}")


Using device: mps


In [7]:
label_map = {
    "No Label": 0,
    "Environment - Energy efficiency" : 11, 
    "Environment - Waste & Pollutants Generation": 12, 
    "Environment - Water Usage": 13, 
    "Environment - Climate Strategy": 14, 
    "Environment - Decarbonisation/Carbon emissions": 15, 
    "Environment - Strategy": 16,
    "Social - Gender and Ethnic Diversity":20,
    "Social - Labor Practices": 21,
    "Social - Human Rights": 22, 
    "Social - Human Capital Management": 23,
    "Social - Occupational Health & Safety":24,
    "Social - Financial Inclusion": 25,           
    "Social - Customer Relations": 26,
    "Social - Privacy Protection": 27,
    "Social - Community investment": 28,
    "Social - Human Capital Management": 29,
    "Governance - Board Diversity":30,
    "Governance - Transparency & Reporting": 31, 
    "Corporate Governance": 32, 
    "Governance - Materiality": 33, 
    "Governance - Risk & Crisis Management": 34,              
    "Governance - Business Ethics": 35, 
    "Governance - Policy Influence": 36, 
    "Governance - Tax Strategy": 37, 
    "Governance - Information Security/ Cybersecurity & System Availability": 38, 
    "Governance - Sustainable Finance": 39,
    "Governance - Shareholder rights": 40
}

def map_labels(label_list):
    if isinstance(label_list, list):  
        return [label_map[label] for label in label_list if label in label_map]
    return []  

df['label'] = df['labels'].apply(map_labels)


In [8]:
df['label'].value_counts()

label
[0]     3587
[29]     827
[31]     620
[26]     617
[33]     487
[11]     438
[30]     412
[39]     404
[25]     360
[24]     321
[27]     303
[12]     293
[35]     287
[13]     283
[28]     253
[14]     225
[34]     192
[36]     172
[15]     110
[38]     107
[40]      93
[22]      92
[16]      80
[20]      66
[37]      59
[32]      12
[21]       2
Name: count, dtype: int64

In [9]:
df.to_csv('../files/label_map_2802.csv', index=False)

In [10]:
empty_rows = df[df['label'].apply(lambda x: len(x) == 0)]
empty_rows

,esg_text,labels,label


## Start training

In [ ]:
df = pd.read_csv('../files/label_map_2802.csv')

In [11]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['esg_text'].values, 
    df['label'].values, 
    test_size=0.2,
    random_state=42
)

In [12]:
len(train_texts)

8561

In [13]:
MODEL_NAME = "nbroad/ESG-BERT"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)  

In [15]:
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = list(map(str, texts))  
        self.labels = list(map(str, labels))  
        self.labels = [int(label[0]) for label in labels]
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        # If we receive a single index
        if isinstance(idx, int):
            text = str(self.texts[idx])
            label = int(self.labels[idx])  
    
            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_length,
                return_token_type_ids=False,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
    
            return {
                'input_ids': encoding['input_ids'].squeeze(0),  # Remove extra dimension
                'attention_mask': encoding['attention_mask'].squeeze(0),
                'labels': torch.tensor(label, dtype=torch.long)
            }
        
        # To receive a list of indices (in case of batching)
        elif isinstance(idx, list):
            batch = [self.__getitem__(i) for i in idx]
            
            # Return batch as dictionary
            return {
                'input_ids': torch.stack([item['input_ids'] for item in batch]),
                'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
                'labels': torch.stack([item['labels'] for item in batch])
            }

# Add into Table

In [ ]:
# import psycopg2
# from datetime import datetime

# # PostgreSQL connection
# conn = psycopg2.connect(
#     dbname="your_database",
#     user="your_user",
#     password="your_password",
#     host="localhost",
#     port="5432"
# )
# cursor = conn.cursor()

# # Example extracted ESG data
# esg_data = {
#     "company_name": "Pfizer",
#     "year": 2023,
#     "carbon_emissions_reduction": 7.5,
#     "renewable_energy_usage": 8.0,
#     "waste_management": 7.0,
#     "workforce_diversity": 6.5,
#     "employee_rights": 8.0,
#     "product_safety": 7.5,
#     "board_independence": 8.5,
#     "transparency": 8.0,
#     "executive_pay_equity": 6.0,
#     "anti_corruption_policies": 7.5,
#     "overall_esg_score": 7.43
# }

# # SQL INSERT query
# query = """
# INSERT INTO ESG_Scores (company_name, year, carbon_emissions_reduction, renewable_energy_usage,
#                         waste_management, workforce_diversity, employee_rights, product_safety,
#                         board_independence, transparency, executive_pay_equity, anti_corruption_policies, 
#                         overall_esg_score, created_at) 
# VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
# """

# # Execute query
# cursor.execute(query, (
#     esg_data["company_name"], esg_data["year"], esg_data["carbon_emissions_reduction"],
#     esg_data["renewable_energy_usage"], esg_data["waste_management"], esg_data["workforce_diversity"],
#     esg_data["employee_rights"], esg_data["product_safety"], esg_data["board_independence"],
#     esg_data["transparency"], esg_data["executive_pay_equity"], esg_data["anti_corruption_policies"],
#     esg_data["overall_esg_score"], datetime.now()
# ))

# # Commit and close
# conn.commit()
# cursor.close()
# conn.close()

# print("ESG data inserted successfully!")
